In [ ]:
TODO:
    
    send_event( event_name, data )
    
    should run state machine in for { event_name, data }

# Example State Machine Usage
## Inter-Machine Communication

In this example, suppose we have two state machines, representing a waiter and cook.

The waiter:

 - Takes customer order
 - Enters Order
 - Waits for Order
 - Gives order to customer
 
The cook:

 - Waits for an order
 - Prepares the order
 - Puts order in "the pass"
 - Waits for another order




In [86]:
from state_machine import StateMachine, State, Model, Condition, AlwaysTrue, AlwaysFalse

In [87]:
class WaiterModel( Model ):
    def __init__( self, name='Bernie' ):
        super( WaiterModel, self ).__init__( name )

waiter_model = WaiterModel()

In [88]:
class CustomerWaves(Condition):
    listens_for='customer'
    
    def __init__(self):
        super(CustomerWaves,self).__init__(name='Customer Waves')
    
    def is_triggered(self, data, model):
        return data=='waves'

CUSTOMER_WAVES = CustomerWaves()

In [89]:
class CustomerOrders(Condition):
    listens_for='customer'
    
    def __init__(self):
        super(CustomerOrders,self).__init__(name='Customer Orders')
        
    def is_triggered(self, data, model):
        return 'orders' in data
            
CUSTOMER_ORDERS = CustomerOrders()

In [90]:
class WhenOrderComplete(Condition):
    listens_for='order_complete'
    
    def __init__(self):
        super(WhenOrderComplete,self).__init__(name='When Order Complete')
        
    def is_triggered(self, data, model):
        return True
    
WHEN_ORDER_COMPLETE = WhenOrderComplete()

In [91]:
class TakeCustomerOrder(State):
    
    def __init__(self):
        super( TakeCustomerOrder, self).__init__(name='Take Customer order')
        
    def on_start(self, data, model, state_input):
        order = { 'order': data['customer']['orders'] }
        model.logger.info( 'Customer has ordered %s'%data['customer']['orders'])
        return order
    
TAKES_CUSTOMER_ORDER = TakeCustomerOrder()

In [92]:
class EnterCustomerOrder(State):
    
    def __init__(self):
        super( EnterCustomerOrder, self).__init__(name='Enters Customer Order')
        
    def on_start(self, data, model, state_input):
        model.logger.info( 'Entering Customer Order %s'%state_input )
        model.logger.info( 'Sending order to listeners:%s'%state_input)
        
        model.send_event( 'order_entered', state_input)
        
ENTER_CUSTOMER_ORDER = EnterCustomerOrder()

In [93]:
class GiveOrderToCustomer(State):
    
    def __init__(self):
        super( GiveOrderToCustomer, self).__init__(name='Give Order to Customer')
        
    def on_start(self, data, model, state_input):
        completed_order = data[ 'order_complete' ]
        model.logger.info( 'Gives %s to customer', completed_order)
    

In [94]:
state_loiter = State('Loitering')
state_wait_for_customer_to_order = State('Wait For Customer to Order')
state_take_customer_order = TAKES_CUSTOMER_ORDER
state_enter_customer_order = ENTER_CUSTOMER_ORDER
state_wait_for_order = State('Wait For Order')
state_give_order_to_customer = GiveOrderToCustomer()

state_loiter.add_transition_to(state_wait_for_customer_to_order, CUSTOMER_WAVES)
state_wait_for_customer_to_order.add_transition_to(state_take_customer_order, CUSTOMER_ORDERS)
state_take_customer_order.add_default_transition_to(state_enter_customer_order)
state_enter_customer_order.add_default_transition_to(state_wait_for_order)
state_wait_for_order.add_transition_to(state_give_order_to_customer,WHEN_ORDER_COMPLETE)

In [95]:
waiter = StateMachine('Waiter')
waiter_model = WaiterModel()
waiter_model.set_state(state_loiter)

data = { 'customer':'waves'}
waiter.run(data, waiter_model)

data = {'customer':{'orders':'pizza'}}
waiter.run(data, waiter_model)

# Simulating the result from the chef
data = {'order_complete':'cooked pizza'}
waiter.run(data, waiter_model)

INFO:default:Testing transition Transition from State:Loitering to State:Wait For Customer to Order, Condition:(Customer Waves)
INFO:default:Returning Transition from State:Loitering to State:Wait For Customer to Order, Condition:(Customer Waves)
INFO:default:Starting state Wait For Customer to Order
INFO:default:Wait For Customer to Order started successfully with result: None
INFO:default:Testing transition Transition from State:Wait For Customer to Order to State:Take Customer order, Condition:(Customer Orders)
INFO:default:Testing transition Transition from State:Wait For Customer to Order to State:Take Customer order, Condition:(Customer Orders)
INFO:default:Returning Transition from State:Wait For Customer to Order to State:Take Customer order, Condition:(Customer Orders)
INFO:default:Starting state Take Customer order
INFO:default:Customer has ordered pizza
INFO:default:Take Customer order started successfully with result: {'order': 'pizza'}
INFO:default:Starting state Enters Cu

In [96]:
class ChefModel(Model):
    
    def __init__(self, name, *args, **kwargs):
        super(ChefModel, self).__init__(name, *args, **kwargs)

In [97]:
class WhenOrderEntered(Condition):
    listens_for='order_entered'
    
    def __init__(self):
        super(WhenOrderEntered,self).__init__(name='When Order Entered')
        
    def is_triggered(self, data, model):
        return True
    
WHEN_ORDER_ENTERED = WhenOrderEntered()

In [98]:
class WhenOrderPrepared(Condition):
    listens_for='time_passes'
    
    def __init__(self):
        super(WhenOrderPrepared,self).__init__(name='When Order Entered')
        
    def is_triggered(self, data, model):
        return True
    
WHEN_ORDER_PREPARED = WhenOrderPrepared()

In [99]:
chef_waits_for_order = State('Chef Waits For Order')
chef_prepares_order = State('Chef Prepares Order')
chef_completes_order = State('Chef Completes Order')

chef_waits_for_order.add_transition_to(chef_prepares_order, WHEN_ORDER_ENTERED)
chef_prepares_order.add_transition_to(chef_completes_order, WHEN_ORDER_PREPARED)

In [100]:
chef = StateMachine('Chef')
chef_model = ChefModel('chef model',state_machine=chef)
chef_model.set_state(chef_waits_for_order)

data = {'order_entered':'pizza'}
chef.run(data,chef_model)

data = {'time_passes':'10 minutes'}
chef.run(data, chef_model)

INFO:default:Testing transition Transition from State:Chef Waits For Order to State:Chef Prepares Order, Condition:(When Order Entered)
INFO:default:Returning Transition from State:Chef Waits For Order to State:Chef Prepares Order, Condition:(When Order Entered)
INFO:default:Starting state Chef Prepares Order
INFO:default:Chef Prepares Order started successfully with result: None
INFO:default:Testing transition Transition from State:Chef Prepares Order to State:Chef Completes Order, Condition:(When Order Entered)
INFO:default:Returning Transition from State:Chef Prepares Order to State:Chef Completes Order, Condition:(When Order Entered)
INFO:default:Starting state Chef Completes Order
INFO:default:Chef Completes Order started successfully with result: None


In [101]:
waiter_model.set_state(state_loiter)
waiter_model.add_listener(chef_model, 'order_entered')

chef_model.set_state(chef_waits_for_order)

data = { 'customer':'waves'}
waiter.run(data, waiter_model)

data = {'customer':{'orders':'pizza'}}
waiter.run(data, waiter_model)

INFO:default:Testing transition Transition from State:Loitering to State:Wait For Customer to Order, Condition:(Customer Waves)
INFO:default:Returning Transition from State:Loitering to State:Wait For Customer to Order, Condition:(Customer Waves)
INFO:default:Starting state Wait For Customer to Order
INFO:default:Wait For Customer to Order started successfully with result: None
INFO:default:Testing transition Transition from State:Wait For Customer to Order to State:Take Customer order, Condition:(Customer Orders)
INFO:default:Testing transition Transition from State:Wait For Customer to Order to State:Take Customer order, Condition:(Customer Orders)
INFO:default:Returning Transition from State:Wait For Customer to Order to State:Take Customer order, Condition:(Customer Orders)
INFO:default:Starting state Take Customer order
INFO:default:Customer has ordered pizza
INFO:default:Take Customer order started successfully with result: {'order': 'pizza'}
INFO:default:Starting state Enters Cu

In [109]:
for x in waiter_model.listeners['order_entered']:
    data = {'order_entered':'pizza'}
    x.state_machine.run( data, x)

INFO:default:Testing transition Transition from State:Chef Waits For Order to State:Chef Prepares Order, Condition:(When Order Entered)
INFO:default:Returning Transition from State:Chef Waits For Order to State:Chef Prepares Order, Condition:(When Order Entered)
INFO:default:Starting state Chef Prepares Order
INFO:default:Chef Prepares Order started successfully with result: None
